In [1]:
# !pip install transformers peft bitsandbytes accelerate trl pydantic-settings scipy codebleu

In [2]:
# DEFINE QUANTIZATION HERE. Choose from ("none" | "8bit" | "4bit")
QUANTIZATION = "4bit"#DO NOT CHANGE
DATASET = "codeparrot/xlcost-text-to-code" #DO NOT CHANGE

In [3]:
#CHANGE THE FOLLOWING
#python, cpp, java, javascript, etc.
MODELTYPE = "BASE_MOE" #BASE or FINETUNED or MOE
model_id = "mistralai/Mistral-7B-v0.1"
PARTITION1 = "Python-program-level" #java-program-level etc.
PARTITION2 = "Java-program-level" #java-program-level etc.
PARTITION3 = "Javascript-program-level" #java-program-level etc.
PARTITION4 = "C++-program-level" #java-program-level etc.

In [4]:
from huggingface_hub import login
login(token="")

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/ec2-user/.cache/huggingface/token
Login successful


In [5]:
import torch
import transformers
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel

In [6]:
from datasets import load_dataset
py_data = load_dataset(DATASET, PARTITION1)
java_data = load_dataset(DATASET, PARTITION2)
js_data = load_dataset(DATASET, PARTITION3)
cpp_data = load_dataset(DATASET, PARTITION4)

In [7]:
print(next(iter(py_data["test"])))
print('\n')
print(next(iter(java_data["test"])))
print('\n')
print(next(iter(js_data["test"])))
print('\n')
print(next(iter(cpp_data["test"])))
print('\n')

{'text': 'Minimum sum possible by removing all occurrences of any array element | Function to find minimum sum after deletion ; Stores frequency of array elements ; Traverse the array ; Calculate sum ; Update frequency of the current element ; Stores the minimum sum required ; Traverse map ; Find the minimum sum obtained ; Return minimum sum ; Input array ; Size of array', 'code': "def minSum ( A , N ) : NEW_LINE INDENT mp = { } NEW_LINE sum = 0 NEW_LINE for i in range ( N ) : NEW_LINE INDENT sum += A [ i ] NEW_LINE if A [ i ] in mp : NEW_LINE INDENT mp [ A [ i ] ] += 1 NEW_LINE DEDENT else : NEW_LINE INDENT mp [ A [ i ] ] = 1 NEW_LINE DEDENT DEDENT minSum = float ( ' inf ' ) NEW_LINE for it in mp : NEW_LINE INDENT minSum = min ( minSum , sum - ( it * mp [ it ] ) ) NEW_LINE DEDENT return minSum NEW_LINE DEDENT arr = [ 4 , 5 , 6 , 6 ] NEW_LINE N = len ( arr ) NEW_LINE print ( minSum ( arr , N ) ) NEW_LINE"}


{'text': 'Minimum sum possible by removing all occurrences of any array elemen

In [8]:
py_testset = py_data["test"]
java_testset = java_data["test"]
js_testset = js_data["test"]
cpp_testset = cpp_data["test"]

In [9]:
print(py_testset)
print(java_testset)
print(js_testset)
print(cpp_testset)

Dataset({
    features: ['text', 'code'],
    num_rows: 887
})
Dataset({
    features: ['text', 'code'],
    num_rows: 911
})
Dataset({
    features: ['text', 'code'],
    num_rows: 886
})
Dataset({
    features: ['text', 'code'],
    num_rows: 909
})


In [10]:
# Pre-define quantization configs

################## 4bit ##################
bb_config_4b_eval = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
##########################################

################## 8bit ##################
bb_config_8b_eval = BitsAndBytesConfig(
    load_in_8bit=True,
)
##########################################

def quantization_config_eval(quantization):
    if quantization == "8bit":
        return bb_config_8b_eval
    else:
        return bb_config_4b_eval

In [11]:
import torch
import transformers
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel

if QUANTIZATION == "none":
    base_model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        trust_remote_code=True,
        use_auth_token=False
    ).to("cuda")
else:
    base_model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=quantization_config_eval(QUANTIZATION),
        device_map="auto",
        trust_remote_code=True,
        use_auth_token=False
    )

tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    model_max_length=512,
    padding_side="left",
    add_eos_token=True)

tokenizer.pad_token = tokenizer.eos_token

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:466: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [01:48<00:00, 54.19s/it]


In [12]:
print(f'Base Model: {model_id}')

Base Model: mistralai/Mistral-7B-v0.1


In [13]:
from torch.nn.utils import parameters_to_vector

total_params = parameters_to_vector(base_model.parameters()).numel()
total_params

3752071168

In [14]:
eval_prompt = f"""<s>[INST] Write a C++ program to complete the following tasks. Minimum sum possible by removing all occurrences of any array element | Function to find minimum sum after deletion ; Stores frequency of array elements ; Traverse the array ; Calculate sum ; Update frequency of the current element ; Stores the minimum sum required ; Traverse map ; Find the minimum sum obtained ; Return minimum sum ; Input array ; Size of array [/INST] \\n </s>"""

In [15]:
torch.cuda.empty_cache()

In [16]:
# eval_prompt = f"""<s>
# For a given integer n, print out all its prime factors one on each line. 
# n = 30
# [INST]
# """
base_model.eval()
with torch.no_grad():
  input_ids = tokenizer(
        eval_prompt,
        return_tensors="pt",
        truncation=True,
        padding="longest",return_attention_mask=True  # Ensure attention mask is returned
    ).input_ids.cuda()
  #input_ids = tokenizer(eval_prompt, return_tensors="pt", truncation=True, return_attention_mask=False).input_ids.cuda()
  outputs = base_model.generate(input_ids=input_ids, max_new_tokens=256, do_sample=True,pad_token_id=tokenizer.eos_token_id, top_p=0.9,temperature=0.8, attention_mask=input_ids.ne(tokenizer.pad_token_id))
  print(f"Prompt:\n{eval_prompt}\n")
  print(f"\nGenerated response:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(eval_prompt):]}")
  print(f'''\nGround truth:{next(iter(js_testset))}''')

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Prompt:
<s>[INST] Write a C++ program to complete the following tasks. Minimum sum possible by removing all occurrences of any array element | Function to find minimum sum after deletion ; Stores frequency of array elements ; Traverse the array ; Calculate sum ; Update frequency of the current element ; Stores the minimum sum required ; Traverse map ; Find the minimum sum obtained ; Return minimum sum ; Input array ; Size of array [/INST] \n </s>


Generated response:
 условие: \n \n [IN] int arr[n], n \n \n [OUT] int minSum \n \n \n \n [CON] -1 \n \n [EX] \n \n [T] 10 \n \n [OUT] 11 \n \n \n \n [EX] \n \n [T] 3 \n \n [OUT] 2 \n \n \n \n [EX] \n \n [T] 2 \n \n [OUT] 1 \n \n \n \n [EX] \n \n [T] 3 \n \n [OUT] 3 \n \n \n \n [EX] \n \n [T] 3 \n \n [OUT] 2 \n \n \n \n [EX] \n \n [T] 3 \n \n [OUT] 1 \n \n \n \n [EX] \n \n [T] 5 \n \n [OUT] 11 \n \n \n \n [EX

Ground truth:{'text': 'Minimum sum possible by removing all occurrences of any array element | Function to find minimum sum after del

## Evaluation

In [17]:
from tqdm import tqdm
import random

EXPERTS = 4
TESTSIZE = 50

# Select first 50 records from each dataset
py_testset = py_data["test"][:TESTSIZE]
java_testset = java_data["test"][:TESTSIZE]
js_testset = js_data["test"][:TESTSIZE]
cpp_testset = cpp_data["test"][:TESTSIZE]

# Combine datasets and retain their labels
combined_data = []
for dataset, label in [(py_testset, 'Python'), (java_testset, 'Java'), (js_testset, 'Javascript'), (cpp_testset, 'C++')]:
    for text, code in zip(dataset['text'], dataset['code']):
        combined_data.append((text, code, label))

# Shuffle combined data
random.shuffle(combined_data)

In [18]:
len(combined_data)

200

In [19]:
import random
from tqdm import tqdm

# Initialize dictionaries for predictions and ground truths
predictions = {}
ground_truths = {}
prompts = {}

count = 0

# Process the shuffled combined data
pbar = tqdm(total=min(0, len(combined_data)), desc="Processing shuffled test datasets")
base_model.eval()
with torch.no_grad():
    for test_prompt, code, label in combined_data:
        if count == TESTSIZE * EXPERTS:
            break
        #torch.cuda.empty_cache()
        # Determine language
        EXPERT = label

        eval_prompt = f"""<s>[INST] Write a {EXPERT} program to complete the following. {test_prompt} [/INST] \\n </s>"""
        ground_truths.setdefault(EXPERT, []).append(code)
        prompts.setdefault(EXPERT,[]).append(eval_prompt)
        input_ids = tokenizer(eval_prompt, return_tensors="pt", truncation=True, padding="longest", return_attention_mask=True).input_ids.cuda()
        outputs = base_model.generate(input_ids=input_ids, max_new_tokens=256, do_sample=True, pad_token_id=tokenizer.eos_token_id, top_p=0.95, temperature=0.8, attention_mask=input_ids.ne(tokenizer.pad_token_id))
        prediction = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
        predictions.setdefault(EXPERT, []).append(prediction[len(eval_prompt):])
        count += 1
        pbar.update(1)

pbar.close()

Processing shuffled test datasets: 0it [00:00, ?it/s]A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Processing shuffled test datasets: 1it [00:06,  6.22s/it]A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Processing shuffled test datasets: 2it [00:24, 13.19s/it]A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Processing shuffled test datasets: 3it [00:42, 15.40s/it]A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Processing shuffled test datasets: 4it [01:00, 16.49s/it]A decoder-only architecture

In [20]:
import pandas as pd

flattened_predictions = [{"expert": expert, "prediction": prediction} for expert, preds in predictions.items() for prediction in preds]

# Create a DataFrame with two columns: "expert" and "prediction"
predictions_df = pd.DataFrame(flattened_predictions)

# Path to the output JSONL file
predictions_output_file = f"{MODELTYPE}_predictions.jsonl"

# Write the DataFrame to the JSONL file
predictions_df.to_json(predictions_output_file, orient='records', lines=True)

In [21]:
predictions_df.head()

,expert,prediction
0,Java,Given below is the main method. \n \n public ...
1,Java,NST] \n \n \n \n [/INST] . #include<stdio.h> #...
2,Java,́ ́ ́ ́ ́ ́ ́ ́ ́ ́ ́ ́ ́ ́ ́ ́ ́ ́ ́ ́ ́ ́ ́...
3,Java,"ужно ввести 2 числа, для определения n и r.\n\..."
4,Java,/\n \n public class Solution {\n public static...


In [22]:
import pandas as pd

flattened_prompts = [{"expert": expert, "prompt": text} for expert, texts in prompts.items() for text in texts]

# Create a DataFrame with two columns: "expert" and "prediction"
prompts_df = pd.DataFrame(flattened_prompts)

# Path to the output JSONL file
prompts_output_file = f"{MODELTYPE}_prompts.jsonl"

# Write the DataFrame to the JSONL file
prompts_df.to_json(prompts_output_file, orient='records', lines=True)

In [23]:
prompts_df.head()

,expert,prompt
0,Java,<s>[INST] Write a Java program to complete the...
1,Java,<s>[INST] Write a Java program to complete the...
2,Java,<s>[INST] Write a Java program to complete the...
3,Java,<s>[INST] Write a Java program to complete the...
4,Java,<s>[INST] Write a Java program to complete the...


In [24]:
import pandas as pd

# Flatten the ground_truths dictionary while preserving the expert name
flattened_ground_truths = [{"expert": expert, "ground_truth": truth} for expert, truths in ground_truths.items() for truth in truths]

# Create a DataFrame with two columns: "expert" and "ground_truth"
references_df = pd.DataFrame(flattened_ground_truths)

# Path to the output JSONL file
references_output_file = f"{MODELTYPE}_ground_truths_{EXPERT}.jsonl"

# Write the DataFrame to the JSONL file
references_df.to_json(references_output_file, orient='records', lines=True)

## CodeBleu

In [54]:
def docodemapping(input):
    if input.lower() == 'java':
        return 'java'
    elif input.lower() == 'python':
        return 'python'
    elif input.lower() == 'c++' or input.lower() == 'cpp':
        return 'cpp'
    elif input.lower() == 'javascript':
        return 'javascript'
    else: 
        print(f'Output not exists: {input}')
        return ""

In [55]:
from codebleu import calc_codebleu
import pandas as pd

results = []
experts = ['Javascript','C++','Java','Python']
for idx, (expert, reference_list) in enumerate(ground_truths.items()):
    if expert in experts:
        for reference, prediction in zip(reference_list, predictions[expert]):
            expert = docodemapping(expert)
            result = calc_codebleu([reference], [prediction], lang=expert, weights=(0.25, 0.25, 0.25, 0.25), tokenizer=None)
            results.append(result)

# Create DataFrame from results
df = pd.DataFrame(results)

# Path to the output CSV file
score_file = f'{MODELTYPE}_eval_report_{EXPERT}.csv'

# Write the DataFrame to the CSV file
df.to_csv(score_file, index=False)

# Display the last result
print(results[-1])

# {
#   'codebleu': 0.5537,
#   'ngram_match_score': 0.1041,
#   'weighted_ngram_match_score': 0.1109,
#   'syntax_match_score': 1.0,
#   'dataflow_match_score': 1.0
# }

/opt/conda/lib/python3.10/site-packages/tree_sitter/__init__.py:36: FutureWarning: Language(path, name) is deprecated. Use Language(ptr, name) instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)


{'codebleu': 0.2683971821259591, 'ngram_match_score': 0.0023890865753850334, 'weighted_ngram_match_score': 0.008699641928451346, 'syntax_match_score': 0.0625, 'dataflow_match_score': 0.0}


In [56]:
df['codebleu'].mean()

0.1660387146986654

In [57]:
report_df = pd.concat([references_df, predictions_df, prompts_df, df], axis="columns")
report_file = f'{MODELTYPE}_eval_full_report_{EXPERT}.csv'
report_df.to_csv(report_file, index=False)

In [58]:
report_df.head()

,expert,ground_truth,expert,prediction,expert,prompt,codebleu,ngram_match_score,weighted_ngram_match_score,syntax_match_score,dataflow_match_score
0,Java,import java . io . * ; class GFG { static int ...,Java,Given below is the main method. \n \n public ...,Java,<s>[INST] Write a Java program to complete the...,0.120482,0.016685,0.023465,0.265306,0.176471
1,Java,import java . util . * ; class GFG { static bo...,Java,NST] \n \n \n \n [/INST] . #include<stdio.h> #...,Java,<s>[INST] Write a Java program to complete the...,0.249804,0.003350,0.005669,0.166667,0.823529
2,Java,class GFG { static boolean check ( int s ) { i...,Java,́ ́ ́ ́ ́ ́ ́ ́ ́ ́ ́ ́ ́ ́ ́ ́ ́ ́ ́ ́ ́ ́ ́...,Java,<s>[INST] Write a Java program to complete the...,0.250000,0.000000,0.000000,0.000000,0.000000
3,Java,class GFG { static int l [ ] [ ] = new int [ 1...,Java,"ужно ввести 2 числа, для определения n и r.\n\...",Java,<s>[INST] Write a Java program to complete the...,0.016121,0.010948,0.010981,0.000000,0.042553
4,Java,import java . io . * ; class GFG { static int ...,Java,/\n \n public class Solution {\n public static...,Java,<s>[INST] Write a Java program to complete the...,0.188441,0.011187,0.016617,0.538462,0.187500


In [ ]:
# from google.colab import files
# files.download(score_file)
# files.download(predictions_output_file)
# files.download(references_output_file)
# files.download(report_file)